In [ ]:
#Use the NBA API to gather player stats from this season as well as last season
%pip install nba_api
%pip install pandas



In [ ]:
from nba_api.stats.endpoints import playercareerstats
from nba_api.stats.static import teams
import pandas as pd

In [ ]:
print(pd.__version__)

In [ ]:
career = playercareerstats.PlayerCareerStats(player_id="203999")

career.get_data_frames()[0]

career.get_json()

career.get_dict()

In [ ]:
nba_teams = teams.get_teams()
#Select dictionary for the Nuggets, which contains their team ID
lakers = [team for team in nba_teams if team['abbreviation'] == 'LAL'][0]
lakers_id = nuggets['id']

In [ ]:
from nba_api.stats.endpoints import leaguegamefinder

#Query for games the Nuggets were playing
gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable = nuggets_id)
#we  want the first dataframe that is returned
games = gamefinder.get_json()
games


AttributeError: 'str' object has no attribute 'head'

In [ ]:
#get the last 10 games that the Nuggets played in 2025 
nuggets_game = games[(games['SEASON_ID'] == '22024') & (games['TEAM_ID'] == nuggets_id)].head(10)






In [ ]:
warriors = [team for team in nba_teams if team['abbreviation'] == 'GSW'][0]
warriors_id = warriors['id']
gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable = warriors_id)
games = gamefinder.get_data_frames()[0]
warriors_games = games[(games['SEASON_ID'] == '22024') & (games['TEAM_ID'] == warriors_id)]




In [ ]:
!pip install openai

In [ ]:
def format_team_stats(team_df):
    "Format team stats for OpenAi"
    wins = team_df['WL'].value_counts()['W']
    losses = team_df['WL'].value_counts()['L']
    average_points = team_df['PTS'].mean()
    win_percentage = (wins/ (wins + losses)) * 100

    stats_summary = f"""
    2025 statistics:
    -Wins and Losses: {wins} - {losses}
    -Average Points scored: {average_points:.1f}
    -Win Percentage: {win_percentage:.1f}%

    Game by Game Results:
    """

    for _, game in team_df.iterrows():
        stats_summary += f"\n{game['GAME_DATE']}: {game['MATCHUP']} - {game['WL']} ({game['PTS']})"

    return stats_summary, team_df['TEAM_NAME']


In [ ]:
import os 
import openai

In [ ]:
openai.api_key  = os.getenv('open_ai_key')

In [ ]:
def get_prediction(team1,team2):
    """Generate a prediction from OpenAI for a game between two teams"""

    prompt = f"""Based on the following NBA statistics, predict the winner and final score of a game between {team1[1][0]} and {team2[1][0]}:
    
    {team1[1][0]} Stats:
    {team1[0]}

    {team2[1][0]} Stats:
    {team2[0]}

    Please provide your prediction in this following format:
    Winner: [team1 name]
    Score: [team2 name][score] - [team2 name][score]
    Confidence: [percentage]
    Brief explanation:[1-2 sentences]

    """

    try:
        response = openai.chat.completions.create(
            model = "gpt-4o-mini",
            messages = [{"role": "system",
                         "content": "You are a professional NBA analyst with expertise in predicting game outcomes based on team statistics"},
                        {"role": "user",
                         "content": prompt}],
                         temperature=0.7
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error getting prediction: {str(e)}"


In [ ]:
nuggets_stats = format_team_stats(nuggets_game)
lakers_stats = format_team_stats(lakers_games)
prediction = get_prediction(nuggets_stats, warriors_stats)

prediction